In [2]:
import os
import parmed as pmd
from Bio.PDB import *
import pdb4amber
import numpy as np
import subprocess
basepath = os.getcwd()
print(os.getcwd())

/scistor/informatica/age206/simlab/US-final/noSASA


In [3]:
def reload_struct(path):
    parser = PDBParser()
    return parser.get_structure('halfpdb', path)

def move_chain(d, chain):
    for res in chain:
        for atom in res:
            newcoord = atom.get_coord()
            newcoord[2] = newcoord[2] + d
            atom.set_coord(newcoord)
    return

def save_struct(name):
    if not name.endswith('.pdb'):
        name += '.pdb'
    io = PDBIO()
    io.set_structure(structure)
    io.save(name)
    return

def generate_rst7(tleap_string, dist):
    with open(f'{dist}/resource/tleap_script.in','w') as f:
        f.write(tleap_string.format(dist))
    subprocess.run(['/cm/shared/package/amber/20_v2/bin/tleap', '-f', f'{dist}/resource/tleap_script.in'])
    return
                   
def write_plumed_file(plumed_string, dist, kappa=150):
    with open(f'{dist}/prod/umb-{dist}.dat', 'w') as f:
        f.write(plumed_string.format(dist, kappa))
    return

def create_folders(dist):
    os.mkdir(str(dist))
    os.mkdir((str(dist) + '/prod'))
    os.mkdir((str(dist) + '/heat'))
    os.mkdir((str(dist) + '/EM'))
    os.mkdir((str(dist) + '/resource'))  
    return

def create_EM_param(em_string, dist):
    with open(f'{dist}/EM/EM.in','w') as f:
        f.write(em_string)
    return

def create_heat_param(heat_string, dist, temp=300.0):
    with open(f'{dist}/heat/heat.in','w') as f:
        f.write(heat_string.format(temp))
    return

def create_prod_param(prod_string, dist, temp=300.0):
    with open(f'{dist}/prod/prod.in','w') as f:
        f.write(prod_string.format(temp, dist))
    return

def convert_pdb(pdb_file, outfile):
    pdb4amber.run(arg_pdbin=pdb_file, arg_pdbout=outfile)
    return

In [4]:
EM_STRING = """IW Minimization
 &cntrl
   imin=1,       ! Run minimization
   ntx=1,        ! Read only coordinates, no velocities
   irest=0,      ! Do not restart
   maxcyc=10000, ! Max 10000 cycles
   ncyc=2000,    ! Steepest descent first 2000 cycles
   ntpr=1000,    ! Update mdout every 1000 cycles
   ntwx=0,       ! No trajectories
   igb=5,        ! GB model
   gbsa=3,       ! SASA is calculated using GPU
   cut=1000,     ! Necesarry for GPU
   saltcon=0.1,  ! salt conc. in M
 /
 """

HEAT_STRING = """IW Heating
 &cntrl
  imin=0,         ! This is an actual MD run
  ntx=1,          ! Read only coordinates, no velocities.
  irest=0,        ! Do not restart.
  nstlim=10000,   ! Number of MD steps (10000*0.002 = 20 ps)
  dt=0.002,       ! Time length of each MD step in ps.
  ntf=2,          ! Do not calculate force for SHAKE constrained bonds
  ntc=2,          ! Allow SHAKE to contrain all bonds
  tempi=0.0,      ! Initial temperature
  temp0={0},    ! Target temperature
  ntpr=100,       ! Print mdout info every 100 cycles.
  ntwx=1000,      ! Write trajectory info every 10000 steps.
  ntb=0           ! No periodic boundary condition
  ntt=3,          ! Langeving thermostat
  gamma_ln=2.0,   ! Colission frequency for langevin
  nmropt=1,       ! NMR restraints
  ig=-1,          ! Randomize the seed
  igb=5,
  gbsa=3,
  saltcon=0.1,
  cut=1000
 /  
&wt type='TEMP0', istep1=0, istep2=9000, value1=0.0, value2={0} /  
&wt type='TEMP0', istep1=9001, istep2=10000, value1={0}, value2={0} /  
&wt type='END' /
"""  

PROD_STRING = """Window run
 &cntrl
  imin=0,  
  ntx=5,  
  irest=1,  
  nstlim=12500000,  ! 25 ns
  dt=0.002,  
  ntf=2,  
  ntc=2,  
  temp0={0},  
  ntpr=2000,  ! Save info every 4 ps
  ntwx=2000,  ! Save traj every 4 ps
  ntb=0,
  ntp=0,    
  ntt=3,  
  barostat=1,  
  gamma_ln=1.0,  
  ig=-1,
  igb=5,
  gbsa=3,
  saltcon=0.1,
  cut=1000,
  plumed=1,
  plumedfile='{1}/prod/umb-{1}.dat'
 /
 """

PLUMED_STRING = """INCLUDE FILE=src/plumed.dat
umbrella: RESTRAINT ARG=dist AT={0} KAPPA={1}

PRINT ARG=* STRIDE=100 FILE={0}/COLVAR
PRINT ARG=dist,energy STRIDE=1 FILE={0}/TIMESERIES"""

TLEAP_STRING = """source leaprc.protein.ff19SB
struct = loadPDB {0}/resource/{0}_cleaned.pdb
setbox struct vdw
saveamberparm struct {0}/resource/{0}_top.parm7 {0}/resource/{0}_start.rst7
quit"""


In [5]:
# Define temps
temps = [280, 300, 320]

# Define distances
old_distances = np.arange(0.3,7.01,0.1).round(1) # First samplings
extra = np.arange(0.61,0.77,0.05).round(2) # Extra sampling in this region
new_distances = np.arange(0.35,7.05,0.1).round(2) # Even more sampling due to unexpected results
distances = np.concatenate([old_distances,extra,new_distances]) # concat everything
distances.sort()

# Store all sims in a nested dict. First dict is temps, second dict is distance: umbrella strength
sims = {temp : {dist: 150 for dist in distances} for temp in temps}
for temp in temps: 
    for x in extra: sims[temp][x] = 300 
    for x in new_distances: sims[temp][x] = 600 if x < 2.0 else 150

{280: {0.3: 150,
  0.35: 600,
  0.4: 150,
  0.45: 600,
  0.5: 150,
  0.55: 600,
  0.6: 150,
  0.61: 300,
  0.65: 600,
  0.66: 300,
  0.7: 150,
  0.71: 300,
  0.75: 600,
  0.76: 300,
  0.8: 150,
  0.85: 600,
  0.9: 150,
  0.95: 600,
  1.0: 150,
  1.05: 600,
  1.1: 150,
  1.15: 600,
  1.2: 150,
  1.25: 600,
  1.3: 150,
  1.35: 600,
  1.4: 150,
  1.45: 600,
  1.5: 150,
  1.55: 600,
  1.6: 150,
  1.65: 600,
  1.7: 150,
  1.75: 600,
  1.8: 150,
  1.85: 600,
  1.9: 150,
  1.95: 600,
  2.0: 150,
  2.05: 150,
  2.1: 150,
  2.15: 150,
  2.2: 150,
  2.25: 150,
  2.3: 150,
  2.35: 150,
  2.4: 150,
  2.45: 150,
  2.5: 150,
  2.55: 150,
  2.6: 150,
  2.65: 150,
  2.7: 150,
  2.75: 150,
  2.8: 150,
  2.85: 150,
  2.9: 150,
  2.95: 150,
  3.0: 150,
  3.05: 150,
  3.1: 150,
  3.15: 150,
  3.2: 150,
  3.25: 150,
  3.3: 150,
  3.35: 150,
  3.4: 150,
  3.45: 150,
  3.5: 150,
  3.55: 150,
  3.6: 150,
  3.65: 150,
  3.7: 150,
  3.75: 150,
  3.8: 150,
  3.85: 150,
  3.9: 150,
  3.95: 150,
  4.0: 150,
  4.05

In [6]:
# Loop over temperatures
for temp in sims.keys():
    
    # Prepare and go to directory
    if not os.path.exists(str(temp)):
        os.mkdir(str(temp))
        os.system(f'cp src/ {temp} -r')
    os.chdir(str(temp))
    print(os.getcwd())
    # Loop over distances 
    for dist, kappa in sims[temp].items():
        
        if os.path.exists(f'{dist}/TIMESERIES'):
            print(f'Skipping {temp}: {dist}')
            continue
        print(f'Now doing run {temp}: {dist}')
        # Create param files
        create_folders(dist)
        create_EM_param(EM_STRING, dist)
        create_heat_param(HEAT_STRING, dist, temp)
        create_prod_param(PROD_STRING, dist, temp)
        write_plumed_file(PLUMED_STRING, dist, kappa)

        # Generate starting config
        structure = reload_struct('src/halfpdb_nosep.pdb')
        chainA = structure[0]['A']
        move_chain(dist * 10, chainA)
        save_struct(f'{dist}/resource/{dist}.pdb')

        # Convert config 
        convert_pdb(f'{dist}/resource/{dist}.pdb', f'{dist}/resource/{dist}_cleaned.pdb')
        generate_rst7(TLEAP_STRING, dist)
        
    # Go back to original dir
    os.chdir(basepath)

/scistor/informatica/age206/simlab/US-final/noSASA/280
Skipping 280: 0.3
Now doing run 280: 0.35
-I: Adding /cm/shared/package/amber/20_v2/dat/leap/prep to search path.
-I: Adding /cm/shared/package/amber/20_v2/dat/leap/lib to search path.
-I: Adding /cm/shared/package/amber/20_v2/dat/leap/parm to search path.
-I: Adding /cm/shared/package/amber/20_v2/dat/leap/cmd to search path.
-f: Source 0.35/resource/tleap_script.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./0.35/resource/tleap_script.in
----- Source: /cm/shared/package/amber/20_v2/dat/leap/cmd/leaprc.protein.ff19SB
----- Source of /cm/shared/package/amber/20_v2/dat/leap/cmd/leaprc.protein.ff19SB done
Log file: ./leap.log
Loading parameters: /cm/shared/package/amber/20_v2/dat/leap/parm/parm19.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc0 + OL3 for RNA + ff19SB
Loading parameters: /cm/shared/package/amber/20_v2/dat/leap/parm/frcmod.ff19SB
Reading force field modification type file (frcmod)
Reading title:

In [7]:
for temp in temps:
    os.chdir(str(temp))
             
    for dist in new_distances:
        command = f'sbatch -J {temp}-{dist} ../batch_window.sh -e -s {dist} -c {dist}/resource/{dist}_start.rst7 -t src/US_topol.parm7'
        print(command)
        
        # Uncomment only when ready to submit!
        # os.system(command)
             
    os.chdir(basepath)

sbatch -J 280-0.35 ../batch_window.sh -e -s 0.35 -c 0.35/resource/0.35_start.rst7 -t src/US_topol.parm7
Submitted batch job 130938
sbatch -J 280-0.45 ../batch_window.sh -e -s 0.45 -c 0.45/resource/0.45_start.rst7 -t src/US_topol.parm7
Submitted batch job 130939
sbatch -J 280-0.55 ../batch_window.sh -e -s 0.55 -c 0.55/resource/0.55_start.rst7 -t src/US_topol.parm7
Submitted batch job 130940
sbatch -J 280-0.65 ../batch_window.sh -e -s 0.65 -c 0.65/resource/0.65_start.rst7 -t src/US_topol.parm7
Submitted batch job 130941
sbatch -J 280-0.75 ../batch_window.sh -e -s 0.75 -c 0.75/resource/0.75_start.rst7 -t src/US_topol.parm7
Submitted batch job 130942
sbatch -J 280-0.85 ../batch_window.sh -e -s 0.85 -c 0.85/resource/0.85_start.rst7 -t src/US_topol.parm7
Submitted batch job 130943
sbatch -J 280-0.95 ../batch_window.sh -e -s 0.95 -c 0.95/resource/0.95_start.rst7 -t src/US_topol.parm7
Submitted batch job 130944
sbatch -J 280-1.05 ../batch_window.sh -e -s 1.05 -c 1.05/resource/1.05_start.rst7 -